In [ ]:
# Import allergen mapping
import json
with open("food_allergen_map.json") as f:
    MAP = json.load(f)

In [5]:
# Split dataset
import os
import shutil

def split_df(data_dir="food-101"):
    with open(f"{data_dir}/meta/test.txt") as f:
        test_list = f.read().splitlines()
    with open(f"{data_dir}/meta/train.txt") as f:
        train_list = f.read().splitlines()

    for split, file_list in zip(["train", "test"], [train_list, test_list]):
        for item in file_list:
            label = item.split("/")[0]
            src = os.path.join(data_dir, "images", item + ".jpg")
            dst_dir = os.path.join("food101_split", split, label)
            os.makedirs(dst_dir, exist_ok=True)
            shutil.copy(src, os.path.join(dst_dir, os.path.basename(item) + ".jpg"))

In [6]:
split_df()

In [7]:
# Check split
train_list = open("food-101/meta/train.txt").read().splitlines()
test_list = open("food-101/meta/test.txt").read().splitlines()

train_len = len(train_list)
test_len = len(test_list)
total = train_len + test_len

train_ratio = train_len / total
test_ratio = test_len / total

print(f"Train images: {train_len}")
print(f"Test images: {test_len}")
print(f"Train/Test Split: {train_ratio:.2f}/{test_ratio:.2f}")

Train images: 75750
Test images: 25250
Train/Test Split: 0.75/0.25


In [9]:
# Train model
from torchvision.models import mobilenet_v3_large
import torch.nn as nn

model = mobilenet_v3_large(pretrained=True)
model.classifier[3] = nn.Linear(model.classifier[3].in_features, 101)

C:\Users\Alin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Alin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Large_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Large_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/mobilenet_v3_large-8738ca79.pth" to C:\Users\Alin/.cache\torch\hub\checkpoints\mobilenet_v3_large-8738ca79.pth


100%|██████████| 21.1M/21.1M [00:06<00:00, 3.25MB/s]


In [14]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [15]:
# Optimizing
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [16]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder("food101_split/train", transform=transform)
test_dataset = datasets.ImageFolder("food101_split/test", transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)


In [18]:
from tqdm import tqdm
import time

EPOCHS = 10

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    start_time = time.time()

    loop = tqdm(enumerate(train_loader), total=len(train_loader), ncols=100)
    loop.set_description(f"Epoch [{epoch+1}/{EPOCHS}]")

    for batch_idx, (images, labels) in loop:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Display live loss info like YOLO
        loop.set_postfix({
            "Batch Loss": f"{loss.item():.4f}",
            "Avg Loss": f"{running_loss / (batch_idx + 1):.4f}"
        })

    epoch_time = time.time() - start_time
    print(f"✅ Epoch {epoch+1} completed in {epoch_time:.2f}s - Avg Loss: {running_loss / len(train_loader):.4f}")


Epoch [1/10]: 100%|█████████| 2368/2368 [56:42<00:00,  1.44s/it, Batch Loss=2.1755, Avg Loss=1.7890]


✅ Epoch 1 completed in 3402.52s - Avg Loss: 1.7890


Epoch [2/10]: 100%|█████████| 2368/2368 [51:48<00:00,  1.31s/it, Batch Loss=2.2172, Avg Loss=1.0438]


✅ Epoch 2 completed in 3108.79s - Avg Loss: 1.0438


Epoch [3/10]: 100%|█████████| 2368/2368 [55:24<00:00,  1.40s/it, Batch Loss=0.2014, Avg Loss=0.7873]


...

✅ Epoch 8 completed in 4012.13s - Avg Loss: 0.2003


Epoch [9/10]: 100%|███████| 2368/2368 [1:06:21<00:00,  1.68s/it, Batch Loss=0.7819, Avg Loss=0.1564]


✅ Epoch 9 completed in 3981.80s - Avg Loss: 0.1564


Epoch [10/10]: 100%|██████| 2368/2368 [1:06:37<00:00,  1.69s/it, Batch Loss=0.9088, Avg Loss=0.1336]

✅ Epoch 10 completed in 3997.83s - Avg Loss: 0.1336


In [19]:
torch.save(model.state_dict(), "mobilenetv3_food101.pth")

In [ ]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Validation Accuracy: {100 * correct / total:.2f}%")


NameError: name 'val_loader' is not defined